<a href="https://colab.research.google.com/github/tanyag7271/fucn_call_base/blob/main/GeminiFunctionCalling_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Gemini Function Calling

This notebook follows along the required code to complete the lab on [How to Interact with APIs Using Function Calling in Gemini
](https://codelabs.developers.google.com/codelabs/gemini-function-calling) by [@koverholt](https://github.com/koverholt).

## Step 1. Overview
This section talks about the importance of Function Calling and how it enables Gemini to access data that may be real-time, protected or otherwise unavailable in the Gemini training datasets.

## Step 2. Setup and requirements

In [ ]:
%%capture
!pip install --upgrade google-cloud-aiplatform

## Step 3. Understand the problem

In [1]:
print("-"*150)

------------------------------------------------------------------------------------------------------------------------------------------------------




*  Task 1: Ask Gemini base version current currency conversion rates using function call
*  Task 2: Enquire about real-time weather conditions of different cities/countries




In [2]:
# Import VertexAI API
import vertexai
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool
)

In [4]:
# call generative model Gemini 1.0

model = GenerativeModel("gemini-1.0-pro")

In [5]:
%%time
response = model.generate_content("What is the meanining of large language models?")
print(response.text)

**Large Language Models (LLMs)**

Large Language Models (LLMs) are advanced machine learning models trained on massive datasets of text. They are neural networks with billions of parameters, enabling them to:

* **Understand and generate human-like text:** LLMs can comprehend and create coherent, grammatically correct, and contextually relevant text.
* **Perform various language tasks:** They excel in tasks such as language translation, question answering, text summarization, code generation, and dialogue generation.
* **Learn from vast amounts of data:** LLMs are trained on datasets containing trillions of words, allowing them to acquire a vast amount of linguistic knowledge.

**Key Features of LLMs:**

* **Self-supervised learning:** Trained on unlabeled text data, LLMs learn to predict the next word in a sequence or to fill in missing words.
* **Generative and predictive:** LLMs can generate new text based on learned patterns and predict the likelihood of word sequences.
* **Context

In [6]:
# LLM is able to answer ques like above, but when asked about real-time or current information, it gives a response with outdated information
response = model.generate_content("what is the weather in paris like today?")
print(response.text)

I do not have real-time capabilities and my knowledge cutoff is April 2023. Therefore, I cannot provide an accurate weather forecast for Paris today.

For the most up-to-date and accurate weather information, I recommend checking a reputable weather app or website, such as the Météo-France website or app.


In [7]:
# In cases like these, you could call an external API then feed the results back to the generative model for it to use in its response
# Step 1: Specify the API
# Step 2: Function Declaration & Tool
# Step 3: Generate a function call on behalf of LLM

In [8]:
import requests
url = "https://api.frankfurter.app/latest"
response = requests.get(url)
response.json()

{'amount': 1.0,
 'base': 'EUR',
 'date': '2024-03-15',
 'rates': {'AUD': 1.6579,
  'BGN': 1.9558,
  'BRL': 5.4461,
  'CAD': 1.4731,
  'CHF': 0.9613,
  'CNY': 7.838,
  'CZK': 25.166,
  'DKK': 7.4571,
  'GBP': 0.8541,
  'HKD': 8.5199,
  'HUF': 393.2,
  'IDR': 17011,
  'ILS': 3.9811,
  'INR': 90.26,
  'ISK': 148.9,
  'JPY': 162.03,
  'KRW': 1448.71,
  'MXN': 18.1915,
  'MYR': 5.1241,
  'NOK': 11.5205,
  'NZD': 1.786,
  'PHP': 60.494,
  'PLN': 4.2953,
  'RON': 4.9711,
  'SEK': 11.2674,
  'SGD': 1.4562,
  'THB': 39.053,
  'TRY': 35.092,
  'USD': 1.0892,
  'ZAR': 20.352}}

In [21]:
# function declaration -----> Tool
get_exchange_rate_func = FunctionDeclaration(
    name="get_exchange_rate",
    description="Get the exchange rate for currencies between countries",
    parameters={
    "type": "object",
    "properties": {
        "currency_date": {
            "type": "string",
            "description": "A date that must always be in YYYY-MM-DD format or the value 'latest' if a time period is not specified"
        },
        "currency_from": {
            "type": "string",
            "description": "The currency to convert from in ISO 4217 format"
        },
        "currency_to": {
            "type": "string",
            "description": "The currency to convert to in ISO 4217 format"
        }
    },
         "required": [
            "currency_from",
            "currency_date",
      ]
  },
)

# register one or more function declarations in a tool
exch_rate_tool = Tool(function_declarations=[get_exchange_rate_func],
                      )

In [23]:
prompt = """What is the current conversion rate from Australian dollars to Indian Rupees?"""

response = model.generate_content(prompt,
                                  tools=[exch_rate_tool],
                                  )


In [24]:
print(response.candidates[0].content)

role: "model"
parts {
  function_call {
    name: "get_exchange_rate"
    args {
      fields {
        key: "currency_date"
        value {
          string_value: "latest"
        }
      }
      fields {
        key: "currency_from"
        value {
          string_value: "AUD"
        }
      }
      fields {
        key: "currency_to"
        value {
          string_value: "INR"
        }
      }
    }
  }
}



In [36]:
params = {}
for key, val in response.candidates[0].content.parts[0].function_call.args.items():
  params[key[9:]] = val
params

{'date': 'latest', 'from': 'AUD', 'to': 'INR'}

In [39]:
url = f"https://api.frankfurter.app/{params['date']}"
api_response = requests.get(url, params = params)
api_response.text

'{"amount":1.0,"base":"AUD","date":"2024-03-15","rates":{"INR":54.442}}'

In [40]:
# create a function response structure to pass that information back to the LLM
response = model.generate_content(
    [
        Content(role="user",
                parts=[Part.from_text(prompt + """Give your answer with context, including exchange rate, date and example taking 10 units of currency to convert from.""")
                ]),
        Content(role="function", parts=[
            Part.from_dict({
                "function_call":{
                     "name":"get_exchange_rate_func",
                     }
                })
            ]),
        Content(role="function", parts=[
            Part.from_function_response(name="get_exchange_rate_func",
                                        response={
                                            "content":api_response.text,
                                            }
                                        )
        ]),
    ],
    tools=[exch_rate_tool]

)

response.candidates[0].content.parts[0].text

'The exchange rate from Australian dollars to Indian Rupees as of 2024-03-15 is 1 AUD = 54.442 INR. Thus, if you exchange 10 Australian dollars, you will get 544.42 Indian Rupees.'